In [1]:
import os
import datetime
import pandas as pd
import re
import great_expectations
import agriculture as ag

In [2]:
#!pip install great_expectations

In [3]:
# ESTE SCRAPPER PUEDE TARDAR, RECOMENDABLE EJECUTAR DIRECTAMENTE DESDE TERMINAL
scrapper = ag.ScrapperMarketAgriculture() # Generamos un objeto de tipo ScrapperMarketAgriculture (scrapper SNIIM)
scrapper.scraping() # Scrapear la pagina del SNIIM para las frutas y hortalizas

Producto: Acelga - Primera
Ya existe registro: acelga_2020.csv
Ya existe registro: acelga_2021.csv
Ya existe registro: acelga_2022.csv
Ya existe registro: acelga_2023.csv
Producto: Aguacate Criollo - Primera
Ya existe registro: aguacate_criollo_2020.csv
Ya existe registro: aguacate_criollo_2021.csv
Ya existe registro: aguacate_criollo_2022.csv
Ya existe registro: aguacate_criollo_2023.csv
Producto: Aguacate Fuerte - Primera
Peticion: {'fechaInicio': '01/01/2020', 'fechaFinal': '01/01/2021', 'ProductoId': '131', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Peticion: {'fechaInicio': '01/01/2021', 'fechaFinal': '01/01/2022', 'ProductoId': '131', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Peticion: {'fechaInicio': '01/01/2022', 'fechaFinal': '01/01/2023', 'ProductoId': '131', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '

Ya existe registro: ciruela_moscatel_2021.csv
Ya existe registro: ciruela_moscatel_2022.csv
Ya existe registro: ciruela_moscatel_2023.csv
Producto: Ciruela Negra - Primera
Ya existe registro: ciruela_negra_2020.csv
Ya existe registro: ciruela_negra_2021.csv
Ya existe registro: ciruela_negra_2022.csv
Ya existe registro: ciruela_negra_2023.csv
Producto: Ciruela pasa - Primera
Ya existe registro: ciruela_pasa_2020.csv
Ya existe registro: ciruela_pasa_2021.csv
Ya existe registro: ciruela_pasa_2022.csv
Ya existe registro: ciruela_pasa_2023.csv
Producto: Ciruela Roja - Primera
Ya existe registro: ciruela_roja_2020.csv
Ya existe registro: ciruela_roja_2021.csv
Ya existe registro: ciruela_roja_2022.csv
Ya existe registro: ciruela_roja_2023.csv
Producto: Coco - Primera
Ya existe registro: coco_2020.csv
Ya existe registro: coco_2021.csv
Ya existe registro: coco_2022.csv
Ya existe registro: coco_2023.csv
Producto: Col grande - Primera
Ya existe registro: col_grande_2020.csv
Ya existe registro: co

Peticion: {'fechaInicio': '01/01/2022', 'fechaFinal': '01/01/2023', 'ProductoId': '297', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Peticion: {'fechaInicio': '01/01/2023', 'fechaFinal': '05/10/2023', 'ProductoId': '297', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Producto: Ejote - Primera
Ya existe registro: ejote_2020.csv
Ya existe registro: ejote_2021.csv
Ya existe registro: ejote_2022.csv
Ya existe registro: ejote_2023.csv
Producto: Ejote largo - Primera
Ya existe registro: ejote_largo_2020.csv
Ya existe registro: ejote_largo_2021.csv
Ya existe registro: ejote_largo_2022.csv
Ya existe registro: ejote_largo_2023.csv
Producto: Ejote redondo - Primera
Ya existe registro: ejote_redondo_2020.csv
Ya existe registro: ejote_redondo_2021.csv
Ya existe registro: ejote_redondo_2022.csv
Ya existe registro: ejote_redondo_2023.csv


Producto: Mandarina Tangerina - Primera
Ya existe registro: mandarina_tangerina_2020.csv
Ya existe registro: mandarina_tangerina_2021.csv
Ya existe registro: mandarina_tangerina_2022.csv
Ya existe registro: mandarina_tangerina_2023.csv
Producto: Mango Ataulfo - Primera
Ya existe registro: mango_ataulfo_2020.csv
Ya existe registro: mango_ataulfo_2021.csv
Ya existe registro: mango_ataulfo_2022.csv
Ya existe registro: mango_ataulfo_2023.csv
Producto: Mango Criollo - Primera
Ya existe registro: mango_criollo_2020.csv
Ya existe registro: mango_criollo_2021.csv
Ya existe registro: mango_criollo_2022.csv
Ya existe registro: mango_criollo_2023.csv
Producto: Mango Haden - Primera
Ya existe registro: mango_haden_2020.csv
Ya existe registro: mango_haden_2021.csv
Ya existe registro: mango_haden_2022.csv
Ya existe registro: mango_haden_2023.csv
Producto: Mango Kent - Primera
Ya existe registro: mango_kent_2020.csv
Ya existe registro: mango_kent_2021.csv
Ya existe registro: mango_kent_2022.csv
Ya ex

Peticion: {'fechaInicio': '01/01/2023', 'fechaFinal': '05/10/2023', 'ProductoId': '513', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Producto: Melón Cantaloupe # 9 - Primera
Ya existe registro: melon_cantaloupe_#_9_2020.csv
Ya existe registro: melon_cantaloupe_#_9_2021.csv
Ya existe registro: melon_cantaloupe_#_9_2022.csv
Ya existe registro: melon_cantaloupe_#_9_2023.csv
Producto: Melón Cantaloupe sin clasificación - Primera
Ya existe registro: melon_cantaloupe_sin_clasificacion_2020.csv
Ya existe registro: melon_cantaloupe_sin_clasificacion_2021.csv
Ya existe registro: melon_cantaloupe_sin_clasificacion_2022.csv
Ya existe registro: melon_cantaloupe_sin_clasificacion_2023.csv
Producto: Membrillo - Primera
Ya existe registro: membrillo_2020.csv
Ya existe registro: membrillo_2021.csv
Ya existe registro: membrillo_2022.csv
Ya existe registro: membrillo_2023.csv
Producto: Nanche - Primera
Ya existe registro:

Peticion: {'fechaInicio': '01/01/2021', 'fechaFinal': '01/01/2022', 'ProductoId': '645', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Peticion: {'fechaInicio': '01/01/2022', 'fechaFinal': '01/01/2023', 'ProductoId': '645', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Peticion: {'fechaInicio': '01/01/2023', 'fechaFinal': '05/10/2023', 'ProductoId': '645', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Producto: Pera Bartlett # 135 - Primera
Peticion: {'fechaInicio': '01/01/2020', 'fechaFinal': '01/01/2021', 'ProductoId': '632', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Peticion: {'fechaInicio': '01/01/2021', 'fechaFinal': '01/01/2022', 'ProductoId': '632', 'Origen

Peticion: {'fechaInicio': '01/01/2021', 'fechaFinal': '01/01/2022', 'ProductoId': '783', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Peticion: {'fechaInicio': '01/01/2022', 'fechaFinal': '01/01/2023', 'ProductoId': '783', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Ya existe registro: sandia_blanca_2023.csv
Producto: Sandía Charleston - Primera
Peticion: {'fechaInicio': '01/01/2020', 'fechaFinal': '01/01/2021', 'ProductoId': '785', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Peticion: {'fechaInicio': '01/01/2021', 'fechaFinal': '01/01/2022', 'ProductoId': '785', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Peticion: {'fechaInicio': '01/01/2022', 'fechaFinal': '

Peticion: {'fechaInicio': '01/01/2021', 'fechaFinal': '01/01/2022', 'ProductoId': '866', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Peticion: {'fechaInicio': '01/01/2022', 'fechaFinal': '01/01/2023', 'ProductoId': '866', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Peticion: {'fechaInicio': '01/01/2023', 'fechaFinal': '05/10/2023', 'ProductoId': '866', 'OrigenId': '-1', 'Origen': 'Todos', 'DestinoId': '-1', 'Destino': 'Todos', 'PreciosPorId': '2', 'RegistrosPorPagina': '5000'}
Producto: Uva sin semilla - Primera
Ya existe registro: uva_sin_semilla_2020.csv
Ya existe registro: uva_sin_semilla_2021.csv
Ya existe registro: uva_sin_semilla_2022.csv
Ya existe registro: uva_sin_semilla_2023.csv
Producto: Uva Superior - Primera
Ya existe registro: uva_superior_2020.csv
Ya existe registro: uva_superior_2021.csv
Ya existe registro:

In [4]:
# Definimos una funcion para obtener el nombre del cultivo a partir del nombre del archivo csv
def estandarizar_nombre(nombre : str) -> str:
    """
    Regresa el nombre del cultivo/producto a partir del nombre del archivo .csv
    """
    cultivo_std = nombre.split('_')[:-1]
    cultivo_std = '_'.join(cultivo_std)
    cultivo_std = re.sub(r"(?<=#)_(?=\d)",'',cultivo_std)
    cultivo_std = re.sub("_"," ",cultivo_std)
    cultivo_std = re.sub("csemilla", "con semilla", cultivo_std)
    cultivo_std = re.sub("ssemilla", "sin semilla", cultivo_std)
    cultivo_std = cultivo_std.title()

    return cultivo_std

In [5]:
# Directorio que contiene los archivos generados por el scrapper
sniim_dir = "./data/sniim/"

In [6]:
# Almacenaremos nuestras dataframes en una lista para despues concatenar
df_list = []
for dirpath, subdir, files in os.walk(sniim_dir):
    for file in files:
        if file == ".DS_Store":
            continue
        
        df = pd.read_csv(os.path.join(dirpath,file))
        df["producto"] = estandarizar_nombre(file)
        df[["destino","central"]] = df["destino"].str.split(": ", expand=True)
        
        cols = df.columns.to_list()
        del cols[-1]
        cols.insert(4,"central")
        
        del cols[-1]
        cols.insert(1,"producto")
        
        df = df[cols]
        
        df_list.append(df)

In [7]:
precios_sniim = pd.concat(df_list,ignore_index=True)

In [8]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec,obs
677545,18/09/2023,Tomate Saladette,Kilogramo,Baja California Sur,Baja California Sur,Unión de Comerciantes de La Paz,23.00,23.0,23.00,NaN
378118,28/03/2023,Nopal Grande,Caja de 20 kg.,Jalisco,Jalisco,Mercado de Abasto de Guadalajara,6.50,7.5,6.50,NaN
544567,29/04/2021,Coliflor Mediana,Caja de 16 pzas.,Baja California,Baja California Sur,Unión de Comerciantes de La Paz,23.75,28.13,26.56,NaN
32888,18/05/2020,Chile Poblano,Kilogramo,Jalisco,Baja California Sur,Unión de Comerciantes de La Paz,15.00,25.0,15.00,NaN
204143,18/07/2022,Elote Grande,Docena,Morelos,México,Central de Abasto de Ecatepec,4.17,5.42,5.00,NaN
216181,22/06/2023,Fresa,Caja de 5 kg.,Importación,Jalisco,Mercado de Abasto de Guadalajara,70.00,74.0,70.00,SELECCIONADA
713757,30/06/2021,Uva Thompson,Caja de 8 kg.,Distrito Federal,Veracruz,Mercado Malibrán,56.25,85.0,56.25,NaN
488327,09/12/2020,Cebolla Morada,Kilogramo,Distrito Federal,Campeche,"Mercado ""Pedro Sáinz de Baranda"", Campeche",20.00,21.0,21.00,NaN
405213,27/02/2023,Platano Manzano,Kilogramo,Chiapas,Nuevo León,"Mercado de Abasto ""Estrella"" de San Nicolás de...",28.00,28.0,28.00,NaN
105111,04/02/2022,Chile Serrano,Arpilla de 30 kg.,Sinaloa,DF,Central de Abasto de Iztapalapa DF,14.00,24.0,17.00,NaN


In [9]:
precios_dict = {
    "fecha": "Fecha en la que se levanto la encuesta",
    "product": "Producto al que hace referencia la entrada/muestra",
    "presentacion": "Presentacion del producto al que se hace referencia",
    "origen": "Entidad Federativa de donde proviene el producto",
    "destino": "Entidad Federativa hacia donde llego el producto",
    "central": "Central de abastos/centro de distribucion de donde se obtuvo la informacion",
    "precio_min": "El valor más bajo de la cotización dentro de una muestra",
    "precio_max": "El valor más alto de la cotización dentro de una muestra",
    "precio_frec": "Es el dato que más se repite en la muestra (moda)",
    "obs": "Observaciones encontradas para la muestra"
}

In [10]:
dict_df = pd.DataFrame.from_dict(precios_dict,orient="index")
dict_df

,0
fecha,Fecha en la que se levanto la encuesta
product,Producto al que hace referencia la entrada/mue...
presentacion,Presentacion del producto al que se hace refer...
origen,Entidad Federativa de donde proviene el producto
destino,Entidad Federativa hacia donde llego el producto
central,Central de abastos/centro de distribucion de d...
precio_min,El valor más bajo de la cotización dentro de u...
precio_max,El valor más alto de la cotización dentro de u...
precio_frec,Es el dato que más se repite en la muestra (moda)
obs,Observaciones encontradas para la muestra


In [11]:
print(f"Porcentaje de entradas faltantes en columna 'obs': {100* precios_sniim['obs'].isna().sum() / precios_sniim.shape[0]:.2f}%")

Porcentaje de entradas faltantes en columna 'obs': 96.26%


In [12]:
precios_sniim.drop(columns="obs", inplace=True)

In [13]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec
825199,19/02/2021,Uva Globo,Caja de 16 kg.,Importación,Aguascalientes,Centro Comercial Agropecuario de Aguascalientes,33.13,34.38,33.75
128862,11/09/2023,Brocoli,Kilogramo,Hidalgo,Hidalgo,Central de Abasto de Pachuca,15.00,16.0,15.00
533931,27/07/2023,Guanabana,Kilogramo,Nayarit,Nayarit,Mercado de abasto 'Adolfo López Mateos' de Tepic,23.00,25.0,24.00
571373,11/05/2023,Calabacita Italiana,Kilogramo,Puebla,Chiapas,Central de Abasto de Tuxtla Gutiérrez,13.00,16.0,13.00
500566,06/01/2020,Yerbabuena,Rollo de 5 kg.,Nayarit,Nayarit,Nayarabastos de Tepic,16.60,17.0,16.80
715456,20/08/2021,Ejote,Kilogramo,Chiapas,Chiapas,Central de Abasto de Tuxtla Gutiérrez,18.00,22.0,18.00
790404,15/06/2023,Limon Con Semilla #3,Caja de 16 kg.,Michoacán,Baja California,"Central de Abasto INDIA, Tijuana",28.13,28.13,28.13
871798,19/05/2020,Lechuga Orejona Grande,Pieza,Chiapas,Chiapas,Central de Abasto de Tuxtla Gutiérrez,10.00,12.0,10.00
33643,18/02/2020,Chile Poblano,Arpilla de 20 kg.,Sinaloa,DF,Central de Abasto de Iztapalapa DF,13.00,20.0,18.00
872980,03/11/2020,Lechuga Orejona Grande,Docena,Oaxaca,Oaxaca,Módulo de Abasto de Oaxaca,7.00,8.0,7.00


In [14]:
precios_sniim["origen"].unique()

array(['Nayarit', 'Distrito Federal', 'Oaxaca', 'Importación', 'Sonora',
       'Yucatán', 'Baja California', 'Campeche', 'San Luis Potosí',
       'Chiapas', 'Durango', 'Hidalgo', 'Michoacán', 'Jalisco',
       'Guanajuato', 'Sinaloa', 'Zacatecas', 'México', 'Tamaulipas',
       'Chihuahua', 'Quintana Roo', 'Puebla', 'Coahuila', 'Guerrero',
       'Colima', 'Veracruz', 'Nuevo León', 'Morelos', 'Querétaro',
       'Baja California Sur', 'Aguascalientes', 'Tabasco', 'Nacional'],
      dtype=object)

In [15]:
precios_sniim["destino"] = precios_sniim["destino"].str.strip()
precios_sniim["destino"].unique()

array(['Nayarit', 'Oaxaca', 'Campeche', 'DF', 'México', 'Yucatán',
       'Baja California', 'Coahuila', 'Chiapas', 'Durango', 'Guanajuato',
       'Guerrero', 'Jalisco', 'Michoacán', 'Aguascalientes', 'Zacatecas',
       'Nuevo León', 'Sonora', 'Morelos', 'Quintana Roo', 'Hidalgo',
       'Puebla', 'San Luis Potosí', 'Baja California Sur', 'Veracruz',
       'Chihuahua', 'Tabasco', 'Colima', 'Sinaloa', 'Tamaulipas',
       'Querétaro'], dtype=object)

In [16]:
precios_sniim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982933 entries, 0 to 982932
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   fecha         982933 non-null  object 
 1   producto      982933 non-null  object 
 2   presentacion  982933 non-null  object 
 3   origen        982933 non-null  object 
 4   destino       982933 non-null  object 
 5   central       982933 non-null  object 
 6   precio_min    982933 non-null  float64
 7   precio_max    982933 non-null  object 
 8   precio_frec   982933 non-null  float64
dtypes: float64(2), object(7)
memory usage: 67.5+ MB


In [17]:
precios_sniim["fecha"] = pd.to_datetime(precios_sniim["fecha"])

In [18]:
precios_sniim["precio_max"] = precios_sniim["precio_max"].str.replace(",","")
precios_sniim["precio_max"] = precios_sniim["precio_max"].astype(float)